In [65]:
import re
import pickle
import os
import string
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [66]:
# Taking query input
query = input("Type the query: ")
# Preprocessing the query to remove punctuations
query = query.lower()
query=query.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
query=query.translate(str.maketrans("‘’’–——−",'       '))
print("processed query",query)

Type the query: I think, I like branded stuff more exclusively than national security
processed query i think  i like branded stuff more exclusively than national security


In [68]:
# Tokenizing the query
query_dict ={}
query_words = []
query_words = word_tokenize(query)
query_words=list(set(words))
# Setting index to 0
query_dict[0]=word_tokenize(query) 
query_dict.items()

dict_items([(0, ['i', 'think', 'i', 'like', 'branded', 'stuff', 'more', 'exclusively', 'than', 'national', 'security'])])

In [69]:
df = pd.read_pickle(r'./Storage/df.pkl')

#Making a copy of the set of words dataframe
buffer = df[1:2]
buffer.values[:]=0
# Indexing at 0
buffer.index=['0']
buffer

,η,branded,abaios,interhemispheric,£208,implementor,colonize,khymia,chymeia,overconnectivity,...,nouria,fontaine,expand,reinhold,daosheng,cadence,coercing,or,sculley,tickets
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
# Populating the query term frequncy dataframe
for doc_id,tokenstream in query_dict.items():
    for token in tokenstream:
        buffer[token]+=1


buffer

C:\Users\vc\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,η,branded,abaios,interhemispheric,£208,implementor,colonize,khymia,chymeia,overconnectivity,...,nouria,fontaine,expand,reinhold,daosheng,cadence,coercing,or,sculley,tickets
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
vocab = pd.read_pickle(r'./Storage/words.pkl')
doc_freq=pd.read_pickle(r'./Storage/doc_freq.pkl')

In [72]:
# Copying the document frequency dictionary and initializing with 0
q_doc_freq = doc_freq.copy()
for word in vocab:
    q_doc_freq[word]=0

In [73]:
doc_freq['branded']

1.8032619709666748

In [84]:
# Populating the query document frequncy dictionary for the words in the query
for word in query_words:
    q_doc_freq[word] = doc_freq[word]

In [75]:
q_doc_freq['branded']

1.8032619709666748

In [76]:
# Vectorizing the query doc frequnecy and calcualting weights
query_vec ={}
query_vec=(1+np.log10(np.array(buffer.loc[:])))*list(q_doc_freq.values())
query_vec[query_vec==-np.inf]=0
query_vec=query_vec/(np.sqrt(sum(query_vec**2)))

query_vec

C:\Users\vc\anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log10
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\vc\anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in multiply
  This is separate from the ipykernel package so we can avoid doing imports until


array([[nan,  1., nan, ..., nan, nan, nan]])

In [77]:
# Convering NaN values to zero
query_vec = np.nan_to_num(query_vec)
query_vec

array([[0., 1., 0., ..., 0., 0., 0.]])

In [78]:
doc_vector = pd.read_pickle('./Storage/doc_vec.pkl')

In [85]:
# Computing scores for the query vector corresponding to each document
scores = {}
for doc_id, sub_vector in doc_vector.items():
    scores[doc_id] = np.sum(np.multiply(query_vec, sub_vector))
#scores

In [80]:
import operator

In [86]:
# Sorting scores in descending order
sorted_scores = dict(sorted(scores.items(), key= operator.itemgetter(1), reverse=True))
#sorted_scores

In [82]:
import itertools

In [83]:
# Returning the top 10 results 
return_docs = itertools.islice(sorted_scores.items(), 10)
for k, v in return_docs:
    print(k)

1505
1488
864
1183
1241
1202
1074
1306
1397
1023
